## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
print(data)
data.info()

       children  days_employed  dob_years education  education_id  \
0             1   -8437.673028         42    высшее             0   
1             1   -4024.803754         36   среднее             1   
2             0   -5623.422610         33   Среднее             1   
3             3   -4124.747207         32   среднее             1   
4             0  340266.072047         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1   -4529.316663         43   среднее             1   
21521         0  343937.404131         67   среднее             1   
21522         1   -2113.346888         38   среднее             1   
21523         3   -3112.481705         38   среднее             1   
21524         2   -1984.507589         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

В столбце days_employed ошибка – отрицательные значения общего трудового стажа.
В столбцах days_employed и total_income, содержащие количественные переменные, имеются пропуски.
Столбецы days_employed и total_income содержат данные вещественного типа вместо целочисленного.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
data['days_employed']=abs(data['days_employed'])

data_days_employed = data.groupby('income_type')['days_employed'].mean()
data_days_employed
        
for income_type in data_days_employed.index:
    data.loc[(data['days_employed'].isnull()) & (data['income_type'] == income_type), 'days_employed'] = data_days_employed.loc[income_type]
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [5]:
data['children'] = data['children'].replace({-1 : 1, 20 : 2})
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [6]:
data_income_type = data.groupby('income_type')['total_income'].mean()
for income_type in data_income_type.index:
    data.loc[(data['total_income'].isnull()) & (data['income_type'] == income_type), 'total_income'] = data_income_type.loc[income_type]
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Мы выявили и обработали пропуски в столбцах days_employed и total_income – теперь в пустых ячейках содержатся средние значения исходя из типа занятости. Пропуски могли возникнуть вследствие наличия в списке студентов, подавших заявку на оплату образования. Однако в студент в списке всего 1, так что гипотеза не подтверждена. 
Вероятно данные были утеряны при выгрузке или произошла ошибка при заносе данных.

### Замена типа данных

In [8]:
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Заменили типы данных столбцов total_income и days_employed, теперь они содержат целочисленные значения. Типы данных были заменены с помощью метода astype(). 

### Обработка дубликатов

In [9]:
data['education'] = data['education'].str.lower()
data.duplicated().sum()

71

In [10]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

### Вывод

Было выявлено 71 дубликат. Значения в столбце education были приведены к нижнему регистру методом lower. Дубликаты удалены с помощью метода drop_deplicates. Проверено их отсутствие.
Дубликаты могли появиться за счет ошибки системы – повторных отправок заявок. 

### Лемматизация

In [25]:
from pymystem3 import Mystem
m = Mystem()

def find_lemmas(text):
    lemmas = ' '.join(m.lemmatize(text)).replace('\n', '')
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    elif ('недвижимость' in lemmas) or ('жилье' in lemmas):
        return 'недвижимость'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    elif 'образование' in lemmas:
        return 'образование'
    else:
        return 'другая категория'
      
data['purpose_lemma'] = data['purpose'].apply(find_lemmas)
data


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,children_category,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,клиент с детьми,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,клиент с детьми,низкий
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,клиент без детей,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,клиент с детьми,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,клиент без детей,средний
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость,клиент с детьми,высокий
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,клиент без детей,средний
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,клиент с детьми,низкий
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,клиент с детьми,высокий


In [26]:
data['purpose_lemma'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_lemma, dtype: int64

### Вывод

Были леммизированы цели получения кредита (столбец 'purpose') в столбец 'purpose_lemma'. Далее преобразовали леммы в категории, в зависимости от цели кредита: автомобиль, недвижимость, свадьба, образование и другая категория. Самая частая цель кредита – покупка недвижимости, 10840 заявок. 


### Категоризация данных

In [27]:
def children_group(row):
    children = row['children']
    if (children > 0):
        return 'клиент с детьми'
    if (children == 0):
        return 'клиент без детей'

row_values = [0]
row_columns = ['children']
row = pd.Series(data=row_values, index=row_columns)
children_group(row)

'клиент без детей'

In [28]:
data['children_category'] = data.apply(children_group, axis=1)

data_children = data.groupby('children_category').agg({'debt': ['sum', 'count']})
data_children['percentage'] = data_children['debt']['sum'] / data_children['debt']['count']*100
data_children

debt        percentage
                    sum  count           
children_category                        
клиент без детей   1063  14091   7.543822
клиент с детьми     678   7363   9.208203

In [29]:
data_children_count = data.groupby('children').agg({'debt': ['sum', 'count']})
data_children_count['percentage'] = data_children_count['debt']['sum'] / data_children_count['debt']['count']*100
data_children_count

debt        percentage
           sum  count           
children                        
0         1063  14091   7.543822
1          445   4855   9.165808
2          202   2128   9.492481
3           27    330   8.181818
4            4     41   9.756098
5            0      9   0.000000

In [30]:
data_family = data.groupby('family_status').agg({'debt': ['sum', 'count']})
data_family['percentage'] = data_family['debt']['sum'] / data_family['debt']['count']*100
data_family

debt        percentage
                       sum  count           
family_status                               
Не женат / не замужем  274   2810   9.750890
в разводе               85   1195   7.112971
вдовец / вдова          63    959   6.569343
гражданский брак       388   4151   9.347145
женат / замужем        931  12339   7.545182

In [31]:
perc33 = data['total_income'].quantile(0.33)
perc67 = data['total_income'].quantile(0.67)

def income_group(income):
    if (income > perc33) and (income < perc67):
        return 'средний'
    elif income > perc67:
        return 'высокий'
    elif (income < perc33) and (income >=0):
        return 'низкий'

data['income_group'] = data['total_income'].apply(income_group)

data_income = data.groupby('income_group').agg({'debt': ['sum', 'count']})
data_income['percentage'] = data_income['debt']['sum'] / data_income['debt']['count']*100
data_income

debt       percentage
              sum count           
income_group                      
высокий       521  7080   7.358757
низкий        578  7080   8.163842
средний       642  7294   8.801755

In [32]:
data_purpose = data.groupby('purpose_lemma').agg({'debt': ['sum', 'count']})
data_purpose['percentage'] = data_purpose['debt']['sum'] / data_purpose['debt']['count']*100
data_purpose

debt        percentage
               sum  count           
purpose_lemma                       
автомобиль     403   4306   9.359034
недвижимость   782  10811   7.233373
образование    370   4013   9.220035
свадьба        186   2324   8.003442

In [33]:
data_gender = data.groupby('gender').agg({'debt': ['sum', 'count']})
data_gender['percentage'] = data_gender['debt']['sum'] / data_gender['debt']['count']*100
data_gender

debt        percentage
        sum  count           
gender                       
F       994  14174   7.012840
M       747   7279  10.262399
XNA       0      1   0.000000

In [34]:
data_education = data.groupby('education').agg({'debt': ['sum', 'count']})
data_education['percentage'] = data_education['debt']['sum'] / data_education['debt']['count']*100
data_education

debt        percentage
                      sum  count           
education                                  
высшее                278   5250   5.295238
начальное              31    282  10.992908
неоконченное высшее    68    744   9.139785
среднее              1364  15172   8.990245
ученая степень          0      6   0.000000

In [35]:
have_debt = data[data['debt'] == 1]['debt'].count()
have_debt_part = have_debt / len(data) *100
have_debt_part

8.115036822970076

In [36]:
data['income_type'].value_counts()

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [37]:
data_income_type = data.groupby('income_type').agg({'debt': ['sum', 'count']})
data_income_type['percentage'] = data_income_type['debt']['sum'] / data_income_type['debt']['count']*100
data_income_type

debt         percentage
                  sum  count            
income_type                             
безработный         1      2   50.000000
в декрете           1      1  100.000000
госслужащий        86   1457    5.902539
компаньон         376   5078    7.404490
пенсионер         216   3829    5.641160
предприниматель     0      2    0.000000
сотрудник        1061  11084    9.572357
студент             0      1    0.000000

### Вывод

Категоризация данных была произведена с помощью создания функций одной строки и классификации по уровню дохода, сформирована таблица для просмотра доли задолжников в зависимости от категории. Исходя из полученных данных, можно сделать следующие выводы.

Средний процент задолжностей составляет 8,1%.

Чаще всего недобросовестные заемщики соответствуювуют следующим критериям:
- имеют детей (в особенности, если количество детей составляет 2 или 4);
- состоят в гражданском браке либо не женаты/замужем;
- имеют средний уровень дохода (от 122 162 до 171 861 условных единиц в месяц);
- берут кредит на приобретение автомобиля или оплату образования;
- имеют начальное образование;
- мужского пола.

Чаще всего выплачивают в срок заемщики, соответствующие следующим критериям:
- не имеют детей;
- вдовец/вдова;
- имеют высокий уровень дохода (171 861 условных единиц в месяц);
- берут кредит на приобретение недвижимости;
- имеют высшее образование (все клиенты с ученой степенью возвращали кредит в срок, но их количество слишком мало, чтобы высчитать их общий процент задолженности);
- женского пола.

Так что наш идеальный клиент – это бездетная вдова с высшим образованием и большим доходом, приобретающая недвижимость :)

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Да. Бездетные заемщики возвращают кредит в срок чаще, чем заемщики с детьми. В особенности это касается клиентов, у которых 2 или 4 ребенка. Процент должников среди этой категории составляет 9,5% и 9,8% соответственно.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Да. Заемщики в гражданском и обычном браке чаще не возвращают кредит в срок по сравнению с заемщиками в другом семейном положении. Процент должников среди этих категорий составляет 9,3% и 9,8% соответственно.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Нет, в целом клиенты возвращают кредит в срок примерно одинаково. Однако заемщики со средним уровнем дохода (от 122 162 до 171 861 условных единиц в месяц) возвращают кредит немного реже остальных. Доля таких заемщиков составляет 8,8%

- Как разные цели кредита влияют на его возврат в срок?

Чаще всего заемщики, взявшие кредит на свадьбу и недвижимость выплачивают кредит вовремя.
Заемщики, взявшие кредит на приобретение автомобиля и оплату образования не выплачивают кредит вовремя чаще остальных – 9,4% и 9,2% соответственно.

### Шаг 4. Общий вывод

Проведен общий анализ статистики платежеспособности клиентов. Задача исследования – выявить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 
Для этого были получены данные клиентов с их общей информацией: количество детей, трудовой стаж, возраст, пол, семейное положение, тип занятости, месячный доход и цель получения кредита.

В ходе обработки данных были выявлено по 2174 пропуска в трудовом стаже и месячном доходе. Это может быть связано с ошибками в выгрузке данных. Также было выявлен 71 дубликат, часть из которых связана с неправильным вводом данных в графе с образованием: что-то написано с большой буквы, что-то только большими буквали, что-то только маленькими. Стоит унифицировать ввод данных об образовании.

Значения трудового стажа и месячного дохода имели не целочисленный формат. Вероятно это связано с ошибкой в программировании при внесении данных в базу.

Сейчас все заявки разделены на 4 вида целей кредита: автомобиль, недвижимость, свадьба и образование. В будущем стоит предлагать именно такой выбор целей для усовершенствования работы с данными. 

Основные выводы:
- средний процент задолженности составляет 8,1%;
- больше половины клиентов составляют сотрудники (11084 заявки), у которых наибольший процент задолженности по сравнению с другими типами занятости – 9,6%;
- большая часть запросов кредита направлена на приобретение недвижимости (10811 заявок). У людей, взявщих кредит с этой целью, процент невозврата кредита самый низкий – 7,2%;
- заемщики, взявшие кредит на приобретение автомобиля и оплату образования не выплачивают кредит в 9,4% и 9,2% случаях соответственно.
- не выевлена зависимость выплаты кредита в срок от уровня дохода, однако клиенты со средним доходом (от 122 162 до 171 861 условных единиц в месяц) не выплачивают кредит вовремя чаще, чем другие клиенты.
- выевлена зависимость выплаты кредита в срок в зависимости от наличия детей. Процент невозврата кредита у бездетных заемщиков составляет 7,5%, в то время как у клиентов с детьми он составляет 9,2%. В особенности это касается клиентов, у которых 2 и 4 ребенка. Процент должников среди этой категории составляет 9,5% и 9,8% соответственно.
- заемщики, состоящие в гражданском и обычном браке, чаще не возвращают кредит в срок по сравнению с другими заемщиками. Процент должников среди этих категорий составляет 9,3% и 9,8% соответственно. В то же время вдовы/вдовцы реже не выплачивают кредит в срок – только 6,6%. 
- проценты невозврата кредита у клиентов с начальным, средним и неоконченным высшим образованием составляют 11%, 9% и 9% соответственно. Клиенты с высшим образованием почти в 2 раза реже не выплачивают кредит в срок – 5,3% случаев. Имеются данные заемщиков с ученой степенью, они все погасили кредит срок. Но поскольку таких клиентов всего 6, нет основания предполагать, что все люди с ученой степенью выплачивают кредит вовремя. 
- также выявлено, что мужчины не выплачивают кредит в срок чаще, чем женщины – 10,3% и 7% соответственно.

Исходя из вышеперечисленного можно сделать вывод, что семейное положение и количество детей влияет на выплату кредита в срок. Клиенты, состоящие в браке или в гражданском браке, а так же имеющие детей, реже выплачивают кредит вовремя. Клиенты, потерявшие своих супругов или в разводе и без детей, являются более добросовестными и не имеют задолженностей чаще. Стоит учесть эту информацию при построении модели кредитного скоринга.